# Исследование надежности заемщиков

Проведем исследование на тему влияния семейного положения, цели кредита, количества детей и дохода на факт погашения кредита в срок. Входные данные - банковские данные клиентов. Результаты исследования могут помочь в построении кредитной политики банка.

Проверим следующие гипотизы:

 1. Есть зависимость между количеством детей и возвратом кредита в срок.
 2. Есть зависимость между семейным положением и возвратом кредита в срок.
 3. Есть зависимость между уровнем дохода и возвратом кредита в срок.
 4. Разные цели кредита влияют на его возврат в срок.

## Откроем таблицу и изучим общую информацию о данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data.csv')
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — days_employed. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs() # заменим все отрицательные значения положительными

In [7]:
data.groupby('income_type')['days_employed'].agg('median') # для каждого типа занятости выведием медианное значение трудового стажа

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)] # удалим аномальные значения

In [10]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [11]:
data.isna().sum() # все пропуски заполнены

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [12]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [13]:
data['education'] = data['education'].str.lower()

In [14]:
data.duplicated().sum()

71

In [15]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [16]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [17]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [18]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [19]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [20]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследуем данные и ответим на вопросы

#### 1) Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [21]:
data.groupby('children')['debt'].sum() #общее количество задолжавших

children
0    1063
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64

In [22]:
data.groupby('children')['debt'].count() #общее количество взявших кредит

children
0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: debt, dtype: int64

In [23]:
data.groupby('children')['debt'].sum() / data.groupby('children')['debt'].count() #отношение выражений показывает долю задолжавших из всех в своей категории по количеству детей

children
0    0.075438
1    0.092346
2    0.094542
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64

**Вывод:** Кредиторы без детей имеют выше шанс отдать кредит по статистике. Кредиторы с 5 детьми тоже хорошо отдают, но выборка слишком мала для вывода на их счет 

#### 2) Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
data.groupby('family_status')['debt'].sum() #группировка по семейному статусу и суммирование задолжавших среди них

family_status
Не женат / не замужем    273
в разводе                 84
вдовец / вдова            63
гражданский брак         385
женат / замужем          927
Name: debt, dtype: int64

In [25]:
data.groupby('family_status')['debt'].count() #группировка по семейному статусу и подсчет общего количества клиентов

family_status
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
гражданский брак          4134
женат / замужем          12261
Name: debt, dtype: int64

In [26]:
data.groupby('family_status')['debt'].sum() / data.groupby('family_status')['debt'].count() #отношение как доля задолжавших среди всех

family_status
Не женат / не замужем    0.097639
в разводе                0.070648
вдовец / вдова           0.066246
гражданский брак         0.093130
женат / замужем          0.075606
Name: debt, dtype: float64

**Вывод:** 
Не женатые и в гражданском браке имеют схожие показатели в 9.5% не рассчитвающихся в срок, на их фоне женатые(7.5%) и в разводе(7%) выглядят как более предпочтительные клиенты. 
Вдовы(цы) имеют самые лучшие показати по возврату средств. Среди них 6.6%  не рассчитались в срок.

#### 3) Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
data.groupby('total_income_category')['debt'].sum() #группировка по категориям доходов и суммирование задолжавших среди них

total_income_category
A       2
B     354
C    1353
D      21
E       2
Name: debt, dtype: int64

In [28]:
data.groupby('total_income_category')['debt'].count() #группировка по категориям доходов и подсчет общего количества клиентов

total_income_category
A       25
B     5014
C    15921
D      349
E       22
Name: debt, dtype: int64

In [29]:
data.groupby('total_income_category')['debt'].sum() / data.groupby('total_income_category')['debt'].count() #отношение как доля катенгорий клиентов, которые допускают задолженности

total_income_category
A    0.080000
B    0.070602
C    0.084982
D    0.060172
E    0.090909
Name: debt, dtype: float64

**Вывод:** Категории А и Е имеют слишком малую выборку, чтобы делать выводы о них. Категория D показывает себя как самый надежный клиент с 6% не рассчитавшимися в срок. Категория В на 1% рассчитывается хуже по сравнению с D, а категория С по отношению D на 2.5% хуже рассчитывается с задолженностью.

#### 4) Как разные цели кредита влияют на его возврат в срок?

In [30]:
data.groupby('purpose_category')['debt'].sum() #группировка по категориям целей и суммирование задолжавших среди них

purpose_category
операции с автомобилем      400
операции с недвижимостью    780
получение образования       369
проведение свадьбы          183
Name: debt, dtype: int64

In [31]:
data.groupby('purpose_category')['debt'].count() #группировка по категориям целей и подсчет общего количества клиентов

purpose_category
операции с автомобилем       4279
операции с недвижимостью    10751
получение образования        3988
проведение свадьбы           2313
Name: debt, dtype: int64

In [32]:
data.groupby('purpose_category')['debt'].sum() / data.groupby('purpose_category')['debt'].count() #отношение как доля клиентов, которые допускают задолженности

purpose_category
операции с автомобилем      0.093480
операции с недвижимостью    0.072551
получение образования       0.092528
проведение свадьбы          0.079118
Name: debt, dtype: float64

**Вывод:** Клиенты, цели которых связаны с автомобилем или образованием хуже всего рассчитываются по кредиту. 9.3% среди них не рассчитываются в срок. 
По цели недвижимость клиенты самые надежные с 7.3% задолжавших клиентов
По цели свадьба 8% не рассчитываются в срок.

#### 5) Bозможные причины появления пропусков в исходных данных.

Пропуски в столбцах: days_employed - 2174, total_income - 2174, - говорят о том, что, вероятнее всего, это ошибка выгрузки, так как человек врядли мог допустить одинаковое количество ошибок в параллельных столбах в таких количествах.  

### Общий вывод

Было проведено исследование на тему влияния семейного положения, цели кредита, количества детей и дохода на факт погашения кредита в срок. Входные данные - банковские данные клиентов. Результаты исследования могут помочь в построении кредитной политики банка.

Для исследования были сформированы следующие гипотизы:

 1. Есть зависимость между количеством детей и возвратом кредита в срок.
 2. Есть зависимость между семейным положением и возвратом кредита в срок.
 3. Есть зависимость между уровнем дохода и возвратом кредита в срок.
 4. Разные цели кредита влияют на его возврат в срок.

### Гопотеза 1 частично подтверждена:
*для более точных выводов нужна больше выборка по клиентам с 3, 4 и 5 детьми.

 1. Среди клиентов без детей доля имеющих просрочку погашения составляет 7.5%;
 2. Среди клиентов с 1 ребенком доля имеющих просрочку погашения составляет 9.2%;
 3. Среди клиентов с 2 детьми доля имеющих просрочку погашения составляет 9.5%.
 
### Гипотеза 2 подтверждена: 

 1. Среди клиентов *в разводе* доля имеющих просрочку погашения составляет 7.1%;
 2. Среди *вдовцов* доля имеющих просрочку погашения составляет 6.6%;
 3. Среди клиентов *в гражданском браке* доля имеющих просрочку погашения составляет 9.3%;
 4. Среди клиентов *женатых / замужем* доля имеющих просрочку погашения составляет 7.6%;
 
### Гипотеза 3 частично подтверждена:
*для более точных выводов нужна больше выборка по клиентам с уровнем дохода по категориям А(от 1 млн. руб), D(30-50 тыс. руб), E(до 30 тыс. руб.).

 1. Среди клиентов с уровнем дохода категории В(200 тыс.- 1 млн. руб) доля имеющих просрочку погашения составляет 7.1%;
 2. Среди клиентов с уровнем дохода категории С(50 тыс.- 200 тыс. руб) доля имеющих просрочку погашения составляет 8.5%.

### Гипотеза 4 подтверждена: 

 1. Среди клиентов с целью *операции с автомобилем* доля имеющих просрочку погашения составляет 9.4%;
 2. Среди клиентов с целью *операции с недвижимостью* доля имеющих просрочку погашения составляет 7.3%;
 3. Среди клиентов с целью *получение образования* доля имеющих просрочку погашения составляет 9.3%;
 4. Среди клиентов с целью *проведение свадьбы* доля имеющих просрочку погашения составляет 7.9%;

### Общие рекомендации:

 1. Я бы обратил внимание на большую утечку данных, в будущем они могут теряться еще больше и это повредит новым исследованиям.
 2. Во многих данных разница между собой составляет менее 5%%, такого рода различия можно отнести к статичтической погрешности и не делать разных стратегий для групп клиентов с такого рода различиями в показателях. 
 3. Гипотезы 2 и 4 подтверждены. Гипотезы 1 и 3 подтверждены частично и требуют больше данных для более глубокого анализа. 